In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import category_encoders as ce
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [2]:
train = pd.read_csv(r'Escritorio/Orga/train.csv', index_col=0)
test = pd.read_csv(r'Escritorio/Orga/test.csv', index_col=0)

In [3]:
#metrica
def MAE(actual, pred):
    return (np.mean(np.absolute(actual - pred)))

## Preprocesamiento

In [4]:
train = train.fillna(0)
test = test.fillna(0)

In [5]:
train = train.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona', 'precio']]
test = test.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona']]

## Feature Engineering

In [6]:
train['cant_amenities'] = train['usosmultiples'] + train['piscina'] + train['gimnasio']
test['cant_amenities'] = test['usosmultiples'] + test['piscina'] + test['gimnasio']

In [8]:
X = train.drop(['precio'], axis=1)
y = train['precio']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Validation shapes: X={X_val.shape}  y={y_val.shape}")
print(f"Test shape: {test.shape}")

Train shapes: X=(180000, 10) y=(180000,)
Validation shapes: X=(60000, 10)  y=(60000,)
Test shape: (60000, 10)


## XGBoost

In [ ]:
best_xgb_model = XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
best_xgb_model.fit(X_train,y_train)

In [ ]:
XGB_pred_val = best_xgb_model.predict(X_val)

In [ ]:
XGB_mae = MAE(y_val, XGB_pred_val)
print(f"MAE XGB: {XGB_mae:.5f}")

In [ ]:
XGB_pred_test = best_xgb_model.predict(test)

In [ ]:
res = pd.DataFrame(XGB_pred_test, index=test.index, columns=['precio'])
res = res.rename(columns={'precio':'target'})
display(res.head())
res.to_csv("workshop-submission-XGB_2.csv", header=True)